In [22]:
import os
import shutil
import glob
from mmcv import Config
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel

import pandas as pd
import numpy as np
import json

In [33]:
# config file_dir, work_dir 수정 후 사용

# config file 들고오기
cfg = Config.fromfile('/opt/ml/input/code/mmsegmentation/configs/_boostcamp_/_base_/upernet-swin-l-aug.py') # load config file
root='/opt/ml/input/data/mmseg/test'
epoch = 'latest'

# dataset config 수정
cfg.work_dir = '/opt/ml/input/code/work_dirs/upernet-swin-l-focal-aug-v2-f1' # set work_dir
cfg.data.test.img_dir = root
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True
cfg.data.samples_per_gpu = 1
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

In [34]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=cfg.data.samples_per_gpu,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

2022-05-11 05:11:50,935 - mmseg - INFO - Loaded 624 images


In [35]:
model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: /opt/ml/input/code/work_dirs/upernet-swin-l-focal-aug-v2-f1/latest.pth


In [36]:
output = single_gpu_test(model, data_loader)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 624/624, 0.5 task/s, elapsed: 1151s, ETA:     0s

In [37]:
#shape 확인
np.array(output).shape

(624, 512, 512)

In [38]:
train_img_root = glob.glob('/opt/ml/input/data/mmseg/images5/training/*.*')
train_ann_root = sorted(glob.glob('/opt/ml/input/data/mmseg/annotations5/training/*.*'))
images = sorted(glob.glob('/opt/ml/input/data/mmseg/test/*.jpg'))

last_img = train_ann_root[-1].split('/')[-1].split('.')[0]

In [39]:
from PIL import Image
category_names = [
    'Backgroud',
    'General trash',
    'Paper',
    'Paper pack',
    'Metal',
    'Glass',
    'Plastic',
    'Styrofoam',
    'Plastic bag',
    'Battery',
    'Clothing'
    ]

category_colors =[
    [0,0,0],
    [192,0,128],
    [0,128,192],
    [0,128,64],
    [128,0,0],
    [64,0,128],
    [64,0,192],
    [192,128,64],
    [192,192,128],
    [64,64,128],
    [128,0,192]
]

category_list = list()
for category in category_colors:
    category_list.extend(category)

In [43]:

#test image copy
for image in images:
    file_name = int(last_img) + int(image.split('/')[-1].split('.')[0]) + 1
    shutil.copyfile(image, os.path.join('/opt/ml/input/data/mmseg/images_pseudo/training', f"{file_name}.jpg"))


In [44]:
##psuedo labeling png 생성
for idx, predict in enumerate(output):
    file_name = int(last_img) + idx + 1
    a = predict
    a=a.astype(np.uint8)
    img_png = Image.fromarray(a).convert('P')
    img_png.putpalette(category_list)
    img_png.save(os.path.join('/opt/ml/input/data/mmseg/annotations_pseudo/training',f'{file_name}.png'))

In [42]:
## submission파일 만들기
# sample_submisson.csv 열기
submission = pd.read_csv('/opt/ml/input/code/submission/sample_submission.csv', index_col=None)
json_dir = os.path.join("/opt/ml/input/data/test.json")
with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)

input_size = 512
output_size = 256
bin_size = input_size // output_size
		
# PredictionString 대입
for image_id, predict in enumerate(output):
    image_id = datas["images"][image_id]
    file_name = image_id["file_name"]
    
    temp_mask = []
    predict = predict.reshape(1, 512, 512)
    # resize predict to 256, 256
    # reference : https://stackoverflow.com/questions/48121916/numpy-resize-rescale-image
    mask = predict.reshape((1, output_size, bin_size, output_size, bin_size)).max(4).max(2) 
    temp_mask.append(mask)
    oms = np.array(temp_mask)
    oms = oms.reshape([oms.shape[0], output_size*output_size]).astype(int)

    string = oms.flatten()

    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=False)